# Single Particle performance plots


   * [#$p_{T}$ response](#$p_{T}$-response)


In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 6)
stuff = []
f_idx = 0

ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)

# define some utility functions
def newCanvas(name=None, title=None, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title
    canvas = ROOT.TCanvas(name, title)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def draw(plot, options='', text=None):
    c = newCanvas()
    c.cd()
    plot.Draw(options)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()

    return


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


class Sample():
    def __init__(cls, name, label, version=None):
        cls.name = name
        cls.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        cls.histo_filename = '../plots1/histos_{}{}.root'.format(cls.name, version)
        cls.histo_file = ROOT.TFile(cls.histo_filename)


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def drawSame(histograms,
             labels,
             options='',
             norm=False,
             logy=False,
             min_y=None,
             max_y=None,
             text=None,
             y_axis_label=None,
             x_axis_label=None):
    global colors
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2

    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        hist.SetStats(False)
        if norm:
            hist.DrawNormalized('same'+','+options, 1.)
        else:
            if hidx:
                hist.Draw('same'+','+options)
            else:
                hist.Draw(options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    if y_axis_label:
        histograms[0].GetYaxis().SetTitle(y_axis_label)
    if x_axis_label:
            histograms[0].GetXaxis().SetTitle(x_axis_label)

    leg.Draw()
    c.Draw()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    if logy:
        c.SetLogy()
    c.Update()


def drawProfileX(histograms, labels, options=''):
    profiles = [hist.ProfileX() for hist in histograms]
    drawSame(profiles, labels, options)


def drawSeveral(histograms, labels, options='', do_profile=False, miny=None, maxy=None, text=None):
    ydiv = int(math.ceil(float(len(histograms))/2))
    for hidx in range(0, len(histograms)):
        newtext = labels[hidx]
        if text:
            newtext = '{}: {}'.format(labels[hidx], text)
        if do_profile:
            drawAndProfileX(histograms[hidx], miny=miny, maxy=maxy, options=options, do_profile=do_profile, text=newtext)
        else:
            draw(histograms[hidx], options=options, text=newtext)


def drawProfileRatio(prof1, prof2, ymin=None, ymax=None, text=None):
    hist1 = prof1.ProjectionX(uuid.uuid4().hex[:6])
    hist2 = prof2.ProjectionX(uuid.uuid4().hex[:6])
    hist1.Divide(hist2)
    draw(hist1)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")

    if ymin is not None and ymax is not None:
        hist1.GetYaxis().SetRangeUser(ymin, ymax)
    ROOT.gPad.Update()


# mean+-nsigmas*RMS.
def drawGaussFit(histo, nsigmas, min, max):
    minfit = histo.GetMean() - nsigmas*histo.GetRMS()
    maxfit = histo.GetMean() + nsigmas*histo.GetRMS()
    drawGFit(histo, min, max, minfit, maxfit)


# Fit a histogram in the range (minfit, maxfit) with a gaussian and
# draw it in the range (min, max)
def drawGFit(histo, min, max, minfit, maxfit):
    # static int i = 0
    # i++
    # gPad->SetGrid(1,1);
    # gStyle->SetGridColor(15);
    histo.GetXaxis().SetRangeUser(min,max)
    global f_idx
    nameF1 = "g{}".format(f_idx)
    f_idx +=1
    g1 = ROOT.TF1(nameF1,"gaus",minfit,maxfit)
    g1.SetLineColor(2)
    g1.SetLineWidth(2)
    histo.Fit(g1,"R")


def drawGraphsSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None):
    global colors
    c = newCanvas()
    c.cd()
    leg = getLegend()

    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        histograms[hidx].Draw('same'+','+options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2
    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    leg.Draw()
    c.Draw()
    if logy:
        c.SetLogy()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    c.Update()


In [4]:
# %load samples.py
import ROOT
import pandas as pd

version = 'v54'

files = {}
file_keys = {}


class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys


class Sample():
    def __init__(self, name, label, version=None, stype=None):
        self.name = name
        self.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        self.histo_filename = '../plots1/histos_{}{}.root'.format(self.name, version)
        self.histo_file = ROOT.TFile(self.histo_filename, 'r')
        self.type = stype


# sample_names = ['ele_flat2to100_PU0',
#                 'ele_flat2to100_PU200',
#                 'photonPt35_PU0',
#                 'photonPt35_PU200']


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary


class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None

    def get(self):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel is None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            self.instance = self.classtype(name, self.root_file)
        return self.instance


class HPlot:
    def __init__(self, samples, tp_sets, tp_selections, gen_selections):
        self.tp_sets = tp_sets
        self.tp_selections = tp_selections
        self.gen_selections = gen_selections
        self.pus = []
        self.labels_dict = {}

        for sample in samples:
            self.pus.append(sample.label)
            self.labels_dict[sample.type] = sample.type
            
        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])

        self.labels_dict.update(tp_sets)
        self.labels_dict.update(tp_selections)
        self.labels_dict.update(gen_selections)
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})
        
        
    def cache_histo(self,
                    classtype,
                    samples,
                    pus,
                    tps,
                    tp_sels,
                    gen_sels):
        if gen_sels is None:
            gen_sels = [None]

        for sample in samples:
            print sample
            for tp in tps:
                for tp_sel in tp_sels:
                    for gen_sel in gen_sels:
                        print sample, tp, tp_sel, gen_sel
                        self.data = self.data.append({'sample': sample.type,
                                                      'pu': sample.label,
                                                      'tp': tp,
                                                      'tp_sel': tp_sel,
                                                      'gen_sel': gen_sel,
                                                      'classtype': classtype,
                                                      'histo': HProxy(classtype, tp, tp_sel, gen_sel, sample.histo_file)},
                                                     ignore_index=True)

    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None):
        histo = None
        labels = []
        text = ''
        
        
        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += '& (gen_sel == @gen_sel)'
        if sample is not None:
            query += '& (sample == @sample)'
            
            
        histo_df = self.data.query(query)
        
        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)
            return None, None, None
#         print histo_df
        
        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])
#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        # print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get() for his in histo_df['histo'].values]
        return histo, labels, text


# -------------------------------------------------------------------------
sample_ele_flat2to100_PU0 = Sample('ele_flat2to100_PU0', 'PU0', version, stype='ele')
sample_ele_flat2to100_PU200 = Sample('ele_flat2to100_PU200', 'PU200', version, stype='ele')

sample_ele_flat2to100_PU0_B = Sample('ele_flat2to100_PU0', 'PU0', version='v53a', stype='ele v53a')
sample_ele_flat2to100_PU200_B = Sample('ele_flat2to100_PU200', 'PU200', version='v53a', stype='ele v53a')



samples_ele = [sample_ele_flat2to100_PU0, sample_ele_flat2to100_PU200]
samples_ele_B = [sample_ele_flat2to100_PU0_B, sample_ele_flat2to100_PU200_B]

# -------------------------------------------------------------------------

sample_photon_flat8to150_PU0 = Sample('photon_flat8to150_PU0', 'PU0', version)
sample_photon_flat8to150_PU200 = Sample('photon_flat8to150_PU200', 'PU0', version)

samples_photons = [sample_photon_flat8to150_PU0, sample_photon_flat8to150_PU200]


sample_gPt35_PU0 = Sample('photonPt35_PU0', 'Pt35 PU0', version)
sample_gPt35_PU200 = Sample('photonPt35_PU200', 'Pt35 PU200', version)

sample_hadronGun_PU0 = Sample('hadronGun_PU0', 'PU0', version)
sample_hadronGun_PU200 = Sample('hadronGun_PU200', 'PU200', version)


samples_photon = [sample_gPt35_PU0, sample_gPt35_PU200]
samples_hadrons = [sample_hadronGun_PU0, sample_hadronGun_PU200]

sample_nugunrate = Sample('nugun_alleta_pu200', 'PU200', version)
samples_nugunrates = [sample_nugunrate]


from python.selections import tp_rate_selections
from python.selections import tp_match_selections
from python.selections import gen_part_selections
from python.selections import eg_rate_selections
from python.selections import eg_qual_selections

tpsets = {'DEF': 'NNDR',
          'DEFCalib': 'NNDR Calib v1'}

tpset_selections = {}

gen_selections = {}
samples = []


#tpset_selections.update(get_label_dict(tp_rate_selections))
# tpset_selections.update(get_label_dict(tp_match_selections))

# gen_selections.update(get_label_dict(gen_part_selections))


Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU0_v54.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU200_v54.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU0_v54.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU200_v54.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v54.root does not exist


In [5]:
tpsets = {'EG': 'EG'}

tpset_selections = {}

gen_selections = {}
samples = []


#tpset_selections.update(get_label_dict(tp_rate_selections))
tpset_selections.update(get_label_dict(eg_qual_selections))
gen_selections.update(get_label_dict(gen_part_selections))

samples = samples_ele

In [6]:
tpset_selections

{'EGq1': 'q1', 'EGq2': 'q2'}

In [7]:
gen_selections

{'GEN': '',
 'GENEtaA': '|#eta^{GEN}| <= 1.52',
 'GENEtaAB': '|#eta^{GEN}| <= 1.7',
 'GENEtaABC': '|#eta^{GEN}| <= 2.4',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaBCDE': '1.52 < |#eta^{GEN}|',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENEtaE': '|#eta^{GEN}| > 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV'}

## Resolution Studies
### 3D cluster: Pt resolution

In [44]:
files

{}

In [11]:
%%time
samples = samples_ele
# samples+=samples_ele_B


hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)


hplot.cache_histo(classtype=histos.EGResoHistos, 
                  samples=samples,
                  pus=[],
                  tps=tpsets,
                  tp_sels=tpset_selections,
                  gen_sels=gen_selections)


# print samples
# for sample in samples:
#     print sample
#     for tp in tpsets:
#         for tp_sel in tpset_selections:
#             for gen_sel in gen_selections:
#                 print sample, tp, tp_sel, gen_sel
#                 hplot.data = hplot.data.append({'sample': 'ele',
#                                                 'pu': sample.label,
#                                                 'tp': tp,
#                                                 'tp_sel': tp_sel,
#                                                 'gen_sel': gen_sel,
#                                                 'classtype': histos.HistoSetReso,
#                                                 'histo': histos.HistoSetReso('{}_{}_{}'.format(tp, tp_sel, gen_sel), sample.histo_file)}
#                                               , ignore_index=True)


<__main__.Sample instance at 0x11cc6f128>
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENEtaB
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENEtaABC
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENEtaA
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENPt40
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENEtaC
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENEtaAB
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENEtaE
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENEtaD
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENEtaBC
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENPt30
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENPt20
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENEtaBCD
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENPt10
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GENEtaBCDE
<__main__.Sample instance at 0x11cc6f128> EG EGq2 GEN
<__main__.Sample instance at 0x11cc6f128> EG EGq1 GENEtaB
<__main__.Sample instance

In [12]:
hplot.data

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,ele,PU0,EG,EGq2,GENEtaB,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x11cc670e0>
1,ele,PU0,EG,EGq2,GENEtaABC,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x11cea34d0>
2,ele,PU0,EG,EGq2,GENEtaA,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x11ceb2290>
3,ele,PU0,EG,EGq2,GENPt40,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x11ceb2bd8>
4,ele,PU0,EG,EGq2,GENEtaC,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x11cbe7c20>
5,ele,PU0,EG,EGq2,GENEtaAB,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x11ceb2950>
6,ele,PU0,EG,EGq2,GENEtaE,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x11ceb2710>
7,ele,PU0,EG,EGq2,GENEtaD,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x11ceb21b8>
8,ele,PU0,EG,EGq2,GENEtaBC,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x11cccaa70>
9,ele,PU0,EG,EGq2,GENPt30,python.l1THistos.EGResoHistos,<__main__.HProxy instance at 0x11ccd1d88>


In [13]:
hplot.labels_dict

{'EG': 'EG',
 'EGq1': 'q1',
 'EGq2': 'q2',
 'GEN': '',
 'GENEtaA': '|#eta^{GEN}| <= 1.52',
 'GENEtaAB': '|#eta^{GEN}| <= 1.7',
 'GENEtaABC': '|#eta^{GEN}| <= 2.4',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaBCDE': '1.52 < |#eta^{GEN}|',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENEtaE': '|#eta^{GEN}| > 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'PU0': 'PU0',
 'PU200': 'PU200',
 'ele': 'ele',
 'ele v53a': 'ele v53a'}

In [14]:
hfd = hplot.get_histo(histos.HistoSetReso, None, ['PU0', 'PU200'], ['DEF','DEFCalib'], 'Em', 'GEN')

print hfd[0]
print hfd[1]
print hfd[2]




No match found for: pu: ['PU0', 'PU200'], tp: ['DEF', 'DEFCalib'], tp_sel: Em, gen_sel: GEN, classtype: python.l1THistos.HistoSetReso
None
None
None


In [49]:
print hfd[2]


EGId


In [50]:

# samples = samples_ele
# tp_sets = ['DEF', 'DEF_em','DEF_em_calib']
# particles = ['ele', 'eleA', 'eleB', 'eleC', 'eleD','eleABC', 'eleBC']



# labels = [sample.label for sample in samples]


# histo_reso_df = pd.DataFrame(index=tp_sets, columns=particles)
# titles_df = pd.DataFrame(index=tp_sets, columns=particles)
# for tp_set in tp_sets:
#     for particle in particles:
#         histo_reso_df.loc[tp_set][particle] =  [histos.HistoSetReso('{}_{}'.format(tp_set, particle), sample.histo_file).hreso for sample in samples]
#         titles_df.loc[tp_set, particle] = '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])
# histo_reso_df

In [51]:
# dir(histo_reso_df.loc['DEF', 'ele'][0])


In [52]:
# titles_df
# lat = ROOT.TLatex(); 
# lat.SetNDC()
# lat.SetTextSize(0.03)

# lat.SetTextColor(ROOT.kBlack)

# lat.DrawLatex(0.2, 0.75, 'mean: {v:.f}'.format(hist1.GetMean()))
# lat.DrawLatex(0.2, 0.71, '#sigma: {v:.f}'.format(hist1.GetMean()))



#### Matching to the highes pT cluster in DR 0.2 from GEN

In [53]:
# ROOT.gStyle.SetPadTopMargin(0.05)
# ROOT.gStyle.SetPadTopMargin(0.13)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)


In [15]:
tps = ['EG']
tp_select = ['EGq1', 'EGq2']
gen_select = ['GEN', 'GENEtaA', 'GENEtaAB', 'GENEtaABC', 'GENEtaB', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE', 'GENEtaC', 'GENEtaD', 'GENEtaE']


## Angular resolution

In [20]:

for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            drawSame([his.h_phiRes for his in hsets], labels, norm=True, text=text)


In [22]:

for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            drawSame([his.h_etaRes for his in hsets], labels, norm=True, text=text)


In [23]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            drawSame([his.h_drRes for his in hsets], labels, norm=True, text=text)


## $p_{T}$ response

In [27]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            drawSame([his.h_ptResp for his in hsets], labels, norm=True, text=text)


In [28]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            drawSeveral([his.h_ptRespVpt for his in hsets], labels, options="COLZ", do_profile=False, text=text)


In [32]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU200'], tp, tp_sel, gen_sel)            
            plots = []
            plots.append(hsets[0].h_ptRespVpt.ProjectionY(uuid.uuid4().hex[:6], 21, 22))
            plots.append(hsets[0].h_ptRespVpt.ProjectionY(uuid.uuid4().hex[:6], 22, 23))
            drawSame(plots, ['A', 'B'], text=text)
            #drawSeveral([his.h_ptRespVpt for his in hsets], labels, options="COLZ", do_profile=False, text=text)


In [29]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in ['GEN']:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            print labels
            print text
            drawSeveral([his.h_ptRespVeta for his in hsets], labels, options="COLZ", do_profile=False, text=text)


['PU0', 'PU200']
EG, q1
['PU0', 'PU200']
EG, q2


#### All clusters in DR 0.2 from GEN

In [ ]:
histo_resoCone_df = pd.DataFrame(index=tp_sets, columns=particles)
for tp_set in tp_sets:
    for particle in particles:
        histo_resoCone_df.loc[tp_set][particle] =  [histos.HistoSetReso('{}_{}'.format(tp_set, particle), sample.histo_file).hresoCone for sample in samples]
histo_resoCone_df

In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptRes for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptResp for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_n010 for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_n025 for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


We now compare the 2 defitnitions (highest pt and all clusters in cone) for the same PU point

In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset_best = [histo.h_ptRes for histo in histo_reso_df.loc[tp_set, particle]]
        hByTPset_cone = [histo.h_ptRes for histo in histo_resoCone_df.loc[tp_set, particle]]

        drawSame([hByTPset_best[0], hByTPset_cone[0]], ['PU0 best', 'PU0 cone'], norm=True, text=titles_df.loc[tp_set, particle])
        drawSame([hByTPset_best[1], hByTPset_cone[1]], ['PU200 best', 'PU200 cone'], norm=True, text=titles_df.loc[tp_set, particle])


We now show the behavior of the pt residuals vs pt of the GEN particle

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_ptResVpt for histo in histosAlgos_all.loc[algo].hresoCone]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True)
    #print hByAlgo[0].GetName()

In [ ]:
for algo in algos:
    hCl3D_ptResVpt_0 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 1, 5) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_0:
        histo.SetTitle('3D cluster reso (GeV) for 1 < pt < 10 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_0, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hCl3D_ptResVpt_1 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 6, 10) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_1:
        histo.SetTitle('3D cluster reso (GeV) for 10 < pt < 20 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_1, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hCl3D_ptResVpt_2 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 11, 15) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_2:
        histo.SetTitle('3D cluster reso (GeV) for 20 < pt < 30 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_2, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hCl3D_ptResVpt_3 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 16, 20) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_3:
        histo.SetTitle('3D cluster reso (GeV) for 30 < pt < 40 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_3, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hByAlgo = [histo.h_ptResVeta for histo in histosAlgos_all.loc[algo].hresoCone]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True)

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_ptResVnclu for histo in histosAlgos_all.loc[algo].hresoCone]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True)

### Energy resolution


In [ ]:


for algo in algos:
    hByAlgo = [histo.h_energyRes for histo in histosAlgos_all.loc[algo].hreso]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSame(hByAlgo, histosAlgos_all.loc[algo]['labels'], norm=True)



In [ ]:

for s_id in range(0, len(samples)):
    labels_bySample = histosAlgos_all['labels'].apply(lambda x: x[s_id])
    hBySample = [histo.h_energyRes for histo in  histosAlgos_all['hreso'].apply(lambda x: x[s_id])]
    drawSame(hBySample, labels_bySample,'hist',logy=True)


## Postion resolution of 2D clusters

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_xResVlayer for histo in histosAlgos_all.loc[algo].hreso2D]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True, miny=-2, maxy=2)

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_xResVlayer.ProjectionY(uuid.uuid4().hex[:6], 1, 3) for histo in histosAlgos_all.loc[algo].hreso2D]
    for histo in hByAlgo:
        histo.SetTitle('2D cluster x reso (cm) for layer 1 and 3')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hByAlgo, histosAlgos_all.loc[algo]['labels'], norm=True, options='hist')


In [ ]:
for algo in algos:
    hByAlgo = [histo.h_xResVlayer.ProjectionY(uuid.uuid4().hex[:6], 12, 15) for histo in histosAlgos_all.loc[algo].hreso2D]
    for histo in hByAlgo:
        histo.SetTitle('2D cluster x reso (cm) for layer 13 and 15')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hByAlgo, histosAlgos_all.loc[algo]['labels'], norm=True, options='hist')


In [ ]:
for algo in algos:
    hByAlgo = [histo.h_yResVlayer for histo in histosAlgos_all.loc[algo].hreso2D]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True, miny=-2, maxy=2)